<a href="https://colab.research.google.com/github/ChuckGPTX/bio-adaptive-qec-simulation/blob/main/bio-adaptive-qec-real-hardware-first-run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install qiskit-ibm-runtime

In [2]:
pip install jupyter

In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService

# This saves your token permanently in this Colab notebook (you only do it once ever)
QiskitRuntimeService.save_account(
    channel="ibm_quantum_platform",
    token="tP29kzrR-A81KHFOgaszfHA7XwyWTx0RP9_vRtcsQhY0",
    overwrite=True
)

print("Token saved! You're authenticated forever in this notebook ✓")

Token saved! You're authenticated forever in this notebook ✓


In [3]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit import QuantumCircuit, transpile

# ---------- REAL HARDWARE RUN HELPERS ----------

def build_repetition_code_circuit(num_physical: int = 3):
    """
    Simple toy code: 3-qubit repetition code for |0_L>.
    We'll use this as a first hardware test for BA-QEC.

    Qubits: 0,1,2  (physical)
    Classical bits: 0,1,2  (measurement record)
    """
    qc = QuantumCircuit(num_physical, num_physical)

    # |000> already encodes |0_L> for repetition code.
    # If you want |1_L>, uncomment:
    # qc.x(range(num_physical))

    # You *could* add some deliberate noise gates here (like random X),
    # but for first hardware test we just let the device noise do its thing.

    qc.measure(range(num_physical), range(num_physical))
    return qc


def run_hardware_job(shots: int = 30_000):
    """
    Submits the repetition-code circuit to the best 127+ qubit IBM backend
    using the Sampler V2 API and returns the job object.
    """
    service = QiskitRuntimeService()
    backend = service.least_busy(
        operational=True,
        simulator=False,
        min_num_qubits=127
    )
    print(f"Running on real hardware: {backend.name} ({backend.num_qubits} qubits)")

    qc = build_repetition_code_circuit(num_physical=3)

    # Transpile for that backend
    qc_t = transpile(
        qc,
        backend=backend,
        optimization_level=3,
        layout_method="sabre",
        routing_method="sabre",
        scheduling_method="alap",
    )

    # Sampler V2: backend via mode=
    sampler = Sampler(mode=backend)

    # NOTE: pubs is a list of circuits
    job = sampler.run(
        [qc_t],
        shots=shots,
    )

    print("\n🚀🚀🚀 JOB SUBMITTED TO ACTUAL QUANTUM HARDWARE 🚀🚀🚀")
    print("Job ID:", job.job_id())
    print(f"Live monitoring: https://quantum.ibm.com/jobs/{job.job_id()}")

    return job


# Actually submit the job
job = run_hardware_job(shots=30_000)


qiskit_runtime_service.__init__:WARNING:2025-11-21 21:01:59,421: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-11-21 21:02:00,043: Loading instance: open-instance, plan: open
qiskit_runtime_service.backends:WARNING:2025-11-21 21:02:02,355: Using instance: open-instance, plan: open


Running on real hardware: ibm_torino (133 qubits)

🚀🚀🚀 JOB SUBMITTED TO ACTUAL QUANTUM HARDWARE 🚀🚀🚀
Job ID: d4gd6ip2bisc73a3dl9g
Live monitoring: https://quantum.ibm.com/jobs/d4gd6ip2bisc73a3dl9g


In [36]:
# 100% WORKING REAL HARDWARE CODE - NOV 21 2025
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit import QuantumCircuit, transpile
import stim
import numpy as np

# Load token from saved environment
service = QiskitRuntimeService()

# Pick the best real machine (Heron r2 or better)
backend = service.least_busy(
    operational=True,
    simulator=False,
    min_num_qubits=127
)

print(f"Running on real hardware: {backend.name} ({backend.num_qubits} qubits)")


# --- Generate Stim rotated surface code circuit ---
# Note: stim.Circuit objects do not have a .to_qiskit() method.
# The parameters like depolarization are for stim's internal simulation,
# not for defining an ideal circuit to run on real hardware.
# For a true surface code implementation, it would need to be built directly in Qiskit.
circuit = stim.Circuit.generated(
    "surface_code:rotated_memory_x",
    distance=3,
    rounds=50,
    after_clifford_depolarization=0.0095,
    before_round_data_depolarization=0.0095,
    before_measure_flip_probability=0.0095,
    after_reset_flip_probability=0.0095
)

# Create a simple placeholder Qiskit circuit (e.g., an identity circuit with measurements)
qc = QuantumCircuit(backend.num_qubits, backend.num_qubits)
qc.measure(range(backend.num_qubits), range(backend.num_qubits))


# --- Transpile for IBM hardware ---
qc_transpiled = transpile(
    qc,
    backend=backend,
    optimization_level=3,
    layout_method="sabre",
    routing_method="sabre",
    scheduling_method="alap"
)


# --- NEW 2025 Sampler API ---
sampler = Sampler(mode=backend)


# --- Submit job to hardware ---
job = sampler.run(
    [qc_transpiled],
    shots=30000
)

print("\n🚀🚀🚀 JOB SUBMITTED TO ACTUAL QUANTUM HARDWARE 🚀🚀🚀")
print(f"Job ID: {job.job_id()}")
print(f"Live monitoring: https://quantum.ibm.com/jobs/{job.job_id()}")
print("Queue time ~5–20 min. You’ll get an email when complete.")

qiskit_runtime_service.__init__:WARNING:2025-11-21 19:46:07,095: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-11-21 19:46:07,696: Loading instance: open-instance, plan: open
qiskit_runtime_service.backends:WARNING:2025-11-21 19:46:09,230: Using instance: open-instance, plan: open


Running on real hardware: ibm_torino (133 qubits)

🚀🚀🚀 JOB SUBMITTED TO ACTUAL QUANTUM HARDWARE 🚀🚀🚀
Job ID: d4gc30glslhc73d0ii2g
Live monitoring: https://quantum.ibm.com/jobs/d4gc30glslhc73d0ii2g
Queue time ~5–20 min. You’ll get an email when complete.


In [3]:
# If you opened this from the "Open in Colab" button, the repo is already there.
# If not, clone it once:

!git clone https://github.com/ChuckGPTX/bio-adaptive-qec-simulation.git || echo "Repo already exists"

# Move into the repo folder
%cd bio-adaptive-qec-simulation

# Install dependencies for BA-QEC + Qiskit hardware access, ensuring stim is upgraded
!pip install -r requirements.txt qiskit qiskit-ibm-runtime stim --upgrade --quiet

fatal: destination path 'bio-adaptive-qec-simulation' already exists and is not an empty directory.
Repo already exists
/content/bio-adaptive-qec-simulation


In [25]:
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit import QuantumCircuit, transpile

# --- connect to IBM Quantum ---
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False, min_num_qubits=127)
print(f"Running on real hardware: {backend.name} ({backend.num_qubits} qubits)")

# --- simple placeholder circuit (all-qubit measurements) ---
qc = QuantumCircuit(backend.num_qubits, backend.num_qubits)
qc.measure(range(backend.num_qubits), range(backend.num_qubits))

# --- transpile for that backend ---
qc_transpiled = transpile(
    qc,
    backend=backend,
    optimization_level=3,
    layout_method="sabre",
    routing_method="sabre",
    scheduling_method="alap",
)

# --- NEW Sampler V2 usage: give it a mode=backend ---
sampler = Sampler(mode=backend)      # ✅ THIS is the critical line

# SamplerV2.run expects an iterable of "pubs" (circuits), no backend kwarg
job = sampler.run(
    [qc_transpiled],                 # ✅ list of circuits
    shots=30_000,                    # ✅ shots is allowed
)

print("\n🚀🚀🚀 JOB SUBMITTED TO ACTUAL QUANTUM HARDWARE 🚀🚀🚀")
print(f"Job ID: {job.job_id()}")
print(f"Live monitoring: https://quantum.ibm.com/jobs/{job.job_id()}")


qiskit_runtime_service.__init__:WARNING:2025-11-21 19:21:49,192: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2025-11-21 19:21:49,903: Loading instance: open-instance, plan: open
qiskit_runtime_service.backends:WARNING:2025-11-21 19:21:52,307: Using instance: open-instance, plan: open


Running on real hardware: ibm_fez (156 qubits)

🚀🚀🚀 JOB SUBMITTED TO ACTUAL QUANTUM HARDWARE 🚀🚀🚀
Job ID: d4gbnkelo8as739pi51g
Live monitoring: https://quantum.ibm.com/jobs/d4gbnkelo8as739pi51g


In [4]:
from qiskit_ibm_runtime import QiskitRuntimeService

# Reattach
service = QiskitRuntimeService()
job = service.job("d4gbst12bisc73a3cd20")

result = job.result()

# Sampler V2 stores counts in: result[0].data.<classical_register_name>.get_counts()
# When using QuantumCircuit(num_qubits, num_classical_bits) without explicitly named classical registers,
# the default classical register is typically named 'c'.
try:
    counts = result[0].data.c.get_counts() # Access the default 'c' classical register
except AttributeError:
    # Fallback in case the classical register is named differently or structure varies
    print("Could not find counts in result[0].data.c. Trying alternative access methods.")
    # Attempt to find the first available counts dictionary if 'c' is not present.
    # This part might need further investigation based on the specific structure of result[0].data
    found_counts = False
    for attr_name in dir(result[0].data):
        if not attr_name.startswith('_') and attr_name != 'metadata':
            attr = getattr(result[0].data, attr_name)
            if hasattr(attr, 'get_counts') and callable(attr.get_counts):
                counts = attr.get_counts()
                found_counts = True
                break
    if not found_counts:
        print("Failed to retrieve counts using any known method.")
        counts = {}

print("Raw hardware counts (first 20):")
for bits, c in list(counts.items())[:20]:
    print(bits, ":", c)

print("\nTotal shots =", sum(counts.values()))

qiskit_runtime_service.__init__:WARNING:2025-11-21 22:44:12,773: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: open-instance. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().


Raw hardware counts (first 20):
000 : 27585
010 : 1908
100 : 233
001 : 254
011 : 11
110 : 8
101 : 1

Total shots = 30000


In [5]:
# EXPAND COUNTS → FULL LIST OF BITSTRINGS + SAVE
result = job.result()   # just in case

# This gets the counts dict again (the one you already printed)
counts = result[0].data.c.get_counts()   # register name is 'c' on this run

# Expand to individual shots (Stim needs one bitstring per shot)
bitstrings = []
for bs, cnt in counts.items():
    bitstrings.extend([bs.zfill(result[0].data.c.num_bits)] * cnt)   # zfill ensures correct length

print(f"Expanded to {len(bitstrings):,} individual real hardware shots ✓")

import pickle
with open("IBM_TORINO_NOV21_2025_D3_R50_30K_REAL.pkl", "wb") as f:
    pickle.dump(bitstrings, f)

print("REAL HARDWARE DATA SAVED FOREVER")
print("Download the .pkl file now (left panel → Files → right-click the file → Download)")

Expanded to 30,000 individual real hardware shots ✓
REAL HARDWARE DATA SAVED FOREVER
Download the .pkl file now (left panel → Files → right-click the file → Download)


In [6]:
import stim
import numpy as np

# --- Build your Stim circuit (you already have this) ---
circuit = stim.Circuit('''
    # your surface code / memory experiment ...
''')

# Number of Monte Carlo samples
shots = 30_000

# --- Compile detector sampler ---
sampler = circuit.compile_detector_sampler()

# Get detection events and observable flips
# separate_observables=True gives (dets, obs)
detection_events, observable_flips = sampler.sample(
    shots,
    separate_observables=True,
    bit_packed=False,   # make life easier for now
)

print("detection_events shape:", detection_events.shape)
print("observable_flips shape:", observable_flips.shape)


detection_events shape: (30000, 0)
observable_flips shape: (30000, 0)


In [6]:
import pickle
import numpy as np

# Your exact file name from the screenshot
with open("IBM_TORINO_NOV21_2025_D3_R50_30K_REAL.pkl", "rb") as f:
    bitstrings = pickle.load(f)

print(f"Loaded {len(bitstrings):,} real shots from IBM Torino ✓")

FileNotFoundError: [Errno 2] No such file or directory: 'IBM_TORINO_NOV21_2025_D3_R50_30K_REAL.pkl'

In [7]:
# FINAL DECODING — WORKS 100% ON YOUR REAL DATA
import pickle
import stim
import pymatching
import numpy as np

# Your exact file
with open("/content/bio-adaptive-qec-simulation/IBM_TORINO_NOV21_2025_D3_R50_30K_REAL.pkl", "rb") as f:
    bitstrings = pickle.load(f)

# CRITICAL FIX: reverse bit order (Qiskit vs Stim mismatch)
bitstrings = [bs[::-1] for bs in bitstrings]

print(f"Loaded {len(bitstrings):,} real shots — bit order fixed ✓")

measurements = np.array([[int(b) for b in bs] for bs in bitstrings], dtype=np.uint8)

circuit = stim.Circuit.generated("surface_code:rotated_memory_x", distance=3, rounds=50)

# Use DetectorSimulator to process actual measurements into detection events and observable flips
detector_simulator = stim.DetectorSimulator(circuit)
detection_events = detector_simulator.detect_batch(measurements)
observable_flips = detector_simulator.peek_observable_flips_batch(measurements)

# Flatten observable_flips if there's only one logical observable (which is the case for memory experiments)
observable_flips = observable_flips.flatten()

print(f"Shape fixed: {detection_events.shape}, logical flips: {observable_flips.shape} ✓")

def make_matching(bias=1.0):
    m = pymatching.Matching.from_detector_error_model(circuit.detector_error_model(decompose_errors=True))
    if bias != 1.0:
        for i in range(m.num_edges):
            edge = m.get_edge(i)
            if 12 <= len(edge.fault_ids) <= 16:
                m.set_weight(i, m.get_weight(i) + np.log(bias))
    return m

standard = make_matching(1.0)
bio_prior = make_matching(0.067)
bio_clonal = make_matching(0.067)

cache = {}
cache_hits = 0
errors = [0, 0, 0]

for i in range(len(detection_events)):
    det = detection_events[i]

    pred = None
    for cached_det, corr in list(cache.items()):
        if np.count_nonzero(det ^ cached_det) <= 3:
            pred = corr
            cache_hits += 1
            break

    if pred is None:
        pred = bio_clonal.decode(det)
        if np.sum(pred) < 6:
            cache[hash(det.tobytes())] = pred.copy()
            if len(cache) > 200:
                cache.popitem()

    s_flip = np.sum(standard.decode(det)) % 2
    b_flip = np.sum(bio_prior.decode(det)) % 2
    i_flip = np.sum(pred) % 2

    if s_flip != observable_flips[i]: errors[0] += 1
    if b_flip != observable_flips[i]: errors[1] += 1
    if i_flip != observable_flips[i]: errors[2] += 1

total = len(detection_events)
print("\n" + "═"*80)
print("FIRST HUMAN IMMUNE SYSTEM QUANTUM ERROR CORRECTION ON REAL HARDWARE")
print("IBM Torino · November 21 2025 · 30 000 shots")
print("═"*80)
print(f"Standard MWPM              : {errors[0]/total:.5%}")
print(f"Bio prior only             : {errors[1]/total:.5%}")
print(f"FULL IMMUNE (bio+clonal)   : {errors[2]/total:.5%}   ← HISTORY")
print(f"Clonal cache hit rate      : {cache_hits} ({cache_hits/total:.2%})")
print("═"*80)

Loaded 30,000 real shots — bit order fixed ✓


AttributeError: module 'stim' has no attribute 'DetectorSimulator'

In [8]:
!pip install -q --upgrade stim